### LOADING LIBRARIES:

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from datetime import datetime

## LOADING THE LOG FILE INTO DATAFRAME.

In [2]:
import json
dict_main= {}
main_list=[]
with  open('ggevent.log') as f:
    for line in f:
        a=json.loads(line)
        main_list.append(a)
        
df_analysis= pd.DataFrame()
df_analysis['game_id']= list(map(lambda x:x['bottle']['game_id'], main_list))
df_analysis['timestamp']= list(map(lambda x:x['bottle']['timestamp'], main_list))
df_analysis['random']= list(map(lambda x:x['headers']['random'],main_list))
df_analysis['sdkv']= list(map(lambda x:x['headers']['sdkv'],main_list))
df_analysis['debug']= list(map(lambda x:x['headers']['debug'],main_list))
df_analysis['ai5']= list(map(lambda x:x['headers']['ai5'],main_list))
df_analysis['ts']= list(map(lambda x:x['post']['ts'],main_list))
df_analysis['event']= list(map(lambda x:x['post']['event'],main_list))
df_analysis['params']= list(map(lambda x:x['params'], main_list))

In [3]:
df_analysis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171721 entries, 0 to 171720
Data columns (total 9 columns):
game_id      171721 non-null object
timestamp    171721 non-null object
random       17792 non-null object
sdkv         171721 non-null object
debug        0 non-null object
ai5          171721 non-null object
ts           171721 non-null object
event        171721 non-null object
params       171721 non-null object
dtypes: object(9)
memory usage: 13.1+ MB


In [4]:
df_analysis.head()

,game_id,timestamp,random,sdkv,debug,ai5,ts,event,params
0,55107008,2016-05-09 02:00:00.004906,None,7.6,None,8fa683e59c02c04cb781ac689686db07,1462759195259,ggstart,{}
1,55107008,2016-05-09 02:00:00.033775,None,7.6,None,335644267c1d5f04eaea7bc6f51b1861,1462759189745,ggstart,{}
2,55107008,2016-05-09 02:00:00.036721,None,7.6,None,ca3071aad676bc963795a2b09635cdf0,1462768152242,ggstop,{}
3,10655437,2016-05-09 02:00:00.041006,None,7.5.1,None,17412dec7d3d02c9b0b1c3d1c3571c5c,1462759193807,ggstop,{}
4,55107008,2016-05-09 02:00:00.074181,None,7.6,None,f81167c854f1a0c86cab6188f9995824,1462759191063,ggstart,{}


### Removing the columns we don't require for problem statement.

In [6]:
df_analysis= df_analysis.drop(['debug','params','random','sdkv','ts'], axis=1)

### Converting time into datetimeformat


In [7]:
from datetime import datetime
df_analysis.timestamp= df_analysis.timestamp.apply(lambda x:datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f"))

In [8]:
df_analysis.head(2)

,game_id,timestamp,ai5,event
0,55107008,2016-05-09 02:00:00.004906,8fa683e59c02c04cb781ac689686db07,ggstart
1,55107008,2016-05-09 02:00:00.033775,335644267c1d5f04eaea7bc6f51b1861,ggstart


In [25]:
def calc(df_analysis_y):
    df_analysis_y.loc[:,'diff_in_time'] = df_analysis_y.timestamp.shift(-1)- df_analysis_y.timestamp
    df_analysis_y.diff_in_time= df_analysis_y.diff_in_time.fillna(0)
    df_analysis_y.loc[:,'session_time_elapsed']= 0
    df_analysis_y.loc[:,'avg_sessions_poss']= 0
    df_analysis_y.loc[:,'sec_diff_in_ggstop_ggstart']=0
    df_analysis_y.loc[:,'event_shifted']= df_analysis_y.event.shift(-1)
    df_analysis_y.event_shifted = df_analysis_y.event_shifted.fillna(0)
    for index,event, event_shifted,diff_in_time in zip(df_analysis_y.index.values, df_analysis_y.event.values, df_analysis_y.event_shifted.values,df_analysis_y.diff_in_time.values):
        if (event =='ggstop' and event_shifted == 'ggstart' and float(diff_in_time*(10**(-9)))>=30.0):
            df_analysis_y.loc[index, ['avg_sessions_poss']]= 1
            df_analysis_y.loc[index,'sec_diff_in_ggstop_ggstart']= float(diff_in_time*(10**(-9)))

        elif (event =='ggstop' and event_shifted == 'ggstart' and float(diff_in_time*(10**(-9)))<30.0):
            df_analysis_y.loc[index,'sec_diff_in_ggstop_ggstart']= float(diff_in_time*(10**(-9)))
        elif (event =='ggstart' and event_shifted== 'ggstop'):
            df_analysis_y.loc[index,'session_time_elapsed']= float(diff_in_time*(10**(-9)))
    total_time_for_present_session=0
    list_valid_session_times= []
    avg_value_start_stop= np.mean(df_analysis_y.session_time_elapsed.values)
    for index,event,event_shifted,diff_in_time in zip(df_analysis_y.index.values, df_analysis_y.event.values, df_analysis_y.event_shifted.values,df_analysis_y.diff_in_time.values):
        if (event== 'ggstart' and event_shifted== 'ggstop'):
            total_time_for_present_session += float(diff_in_time*(10**(-9)))

        elif (event== 'ggstop' and event_shifted== 'ggstop' and float(diff_in_time*(10**(-9)))>(30.0+avg_value_start_stop)):
            if total_time_for_present_session>60.0:
                df_analysis_y.loc[index,'avg_sessions_poss'] = 1
                list_valid_session_times.append(total_time_for_present_session)
                total_time_for_present_session=0
            else :
                total_time_for_present_session=0
                total_time_for_present_session= avg_value_start_stop

        elif (event=='ggstop' and event_shifted== 'ggstop' and float(diff_in_time*(10**(-9)))<(30.0+avg_value_start_stop)):
            total_time_for_present_session= total_time_for_present_session + float(diff_in_time*(10**(-9)))


        elif (event== 'ggstart' and event_shifted== 'ggstart' and float(diff_in_time*(10**(-9)))>(30.0+avg_value_start_stop)):
            total_time_for_present_session= total_time_for_present_session+ avg_value_start_stop
            if total_time_for_present_session>60.0:
                list_valid_session_times.append(total_time_for_present_session)
                #print(event,event_shifted,total_time_for_present_session,'more')
                total_time_for_present_session=0
                df_analysis_y.loc[index,'avg_sessions_poss'] = 1
            else:
                total_time_for_present_session=0

        elif (event=='ggstart' and event_shifted== 'ggstart' and float(diff_in_time*(10**(-9)))<(30.0+avg_value_start_stop)):
            total_time_for_present_session = total_time_for_present_session + float(diff_in_time*(10**(-9)))/2
            #print(event,event_shifted,total_time_for_present_session)



        elif (event== 'ggstop' and event_shifted== 'ggstart' and float(diff_in_time*(10**(-9)))>30.0):
            if total_time_for_present_session>60.0:
                list_valid_session_times.append(total_time_for_present_session)
                #print(event,event_shifted,total_time_for_present_session,'more')
                total_time_for_present_session=0
            else :
                total_time_for_present_session=0

        if (total_time_for_present_session>60.0 and index== df_analysis_y.index.values[-1]):
            list_valid_session_times.append(total_time_for_present_session)
            #print(event,event_shifted,total_time_for_present_session,'nothing')

    return list_valid_session_times


## Let's test our code with few examples:

In [17]:
print(df_analysis.game_id.unique())
print(len(df_analysis.game_id.unique()))

['55107008' '10655437' '30900473' '18121481' '32950170' '43346372'
 '19049241' '10483946' '40904844' '67544688' '63960431' '21554188'
 '60374084' '98010070' '85159873' '82196685' '94868282' '35643983'
 '26962869' '59561974']
20


In [18]:
print(df_analysis.ai5.unique())
print(len(df_analysis.ai5.unique()))

['8fa683e59c02c04cb781ac689686db07' '335644267c1d5f04eaea7bc6f51b1861'
 'ca3071aad676bc963795a2b09635cdf0' ..., 'e1539460c05843b5ab17478d79edf072'
 '77abb14b6ad93efb332b7553795d3009' 'd5dee568ebd06e1183c6926bc5ec4124']
17419


In [37]:
df_analysis.head(15)

,game_id,timestamp,ai5,event
0,55107008,2016-05-09 02:00:00.004906,8fa683e59c02c04cb781ac689686db07,ggstart
1,55107008,2016-05-09 02:00:00.033775,335644267c1d5f04eaea7bc6f51b1861,ggstart
2,55107008,2016-05-09 02:00:00.036721,ca3071aad676bc963795a2b09635cdf0,ggstop
3,10655437,2016-05-09 02:00:00.041006,17412dec7d3d02c9b0b1c3d1c3571c5c,ggstop
4,55107008,2016-05-09 02:00:00.074181,f81167c854f1a0c86cab6188f9995824,ggstart
5,10655437,2016-05-09 02:00:00.084916,17412dec7d3d02c9b0b1c3d1c3571c5c,ggstart
6,30900473,2016-05-09 02:00:00.093961,a2659df45c8d05f326225fa5b1063ac9,ggstart
7,18121481,2016-05-09 02:00:00.099308,b8bbef76f8dfee2fe190a283cd5a19a7,ggstart
8,18121481,2016-05-09 02:00:00.105905,e8ebfc3f39512eda3aa0702b13ffed63,ggstart
9,55107008,2016-05-09 02:00:00.156922,988e4873861347113519fbee6dd1c3b0,ggstart


set1= ['55107008','335644267c1d5f04eaea7bc6f51b1861']


set2= ['10655437','17412dec7d3d02c9b0b1c3d1c3571c5c']


set3= ['18121481','e8ebfc3f39512eda3aa0702b13ffed63']

In [20]:
df_set1= df_analysis[(df_analysis.game_id=='55107008')&(df_analysis.ai5=='335644267c1d5f04eaea7bc6f51b1861')]

In [26]:
calc(df_set1)

//anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
//anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2387: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


[136.0, 125.0, 659.0, 421.0, 137.0]

In [27]:
df_set1

,game_id,timestamp,ai5,event,diff_in_time,session_time_elapsed,avg_sessions_poss,sec_diff_in_ggstop_ggstart,event_shifted
1,55107008,2016-05-09 02:00:00.033775,335644267c1d5f04eaea7bc6f51b1861,ggstart,00:02:16.372923,136,0,0,ggstop
5180,55107008,2016-05-09 02:02:16.406698,335644267c1d5f04eaea7bc6f51b1861,ggstop,00:02:16.788751,0,1,136,ggstart
10790,55107008,2016-05-09 02:04:33.195449,335644267c1d5f04eaea7bc6f51b1861,ggstart,00:02:05.020421,125,0,0,ggstop
15833,55107008,2016-05-09 02:06:38.215870,335644267c1d5f04eaea7bc6f51b1861,ggstop,00:01:39.092569,0,1,99,ggstart
19808,55107008,2016-05-09 02:08:17.308439,335644267c1d5f04eaea7bc6f51b1861,ggstart,00:00:05.610737,0,0,0,ggstart
20070,55107008,2016-05-09 02:08:22.919176,335644267c1d5f04eaea7bc6f51b1861,ggstart,00:09:03.505822,543,0,0,ggstop
43124,55107008,2016-05-09 02:17:26.424998,335644267c1d5f04eaea7bc6f51b1861,ggstop,00:00:04.750527,0,0,4,ggstart
43347,55107008,2016-05-09 02:17:31.175525,335644267c1d5f04eaea7bc6f51b1861,ggstart,00:07:59.407296,0,1,0,ggstart
64832,55107008,2016-05-09 02:25:30.582821,335644267c1d5f04eaea7bc6f51b1861,ggstart,00:07:01.527036,421,0,0,ggstop
84602,55107008,2016-05-09 02:32:32.109857,335644267c1d5f04eaea7bc6f51b1861,ggstop,00:04:37.308923,0,1,277,ggstart


In [38]:
#set2= ['10483946','ca3071aad676bc963795a2b09635cdf0']
df_set2= df_analysis[(df_analysis.game_id=='10655437')&(df_analysis.ai5=='17412dec7d3d02c9b0b1c3d1c3571c5c')]

In [39]:
df_set2

,game_id,timestamp,ai5,event
3,10655437,2016-05-09 02:00:00.041006,17412dec7d3d02c9b0b1c3d1c3571c5c,ggstop
5,10655437,2016-05-09 02:00:00.084916,17412dec7d3d02c9b0b1c3d1c3571c5c,ggstart
3275,10655437,2016-05-09 02:01:25.783037,17412dec7d3d02c9b0b1c3d1c3571c5c,ggstop


In [40]:
calc(df_set2)

//anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
//anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
//anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2387: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

[85.0]

In [41]:
#set3= ['18121481','e8ebfc3f39512eda3aa0702b13ffed63']
df_set3 = df_analysis[(df_analysis.game_id=='18121481')&(df_analysis.ai5=='e8ebfc3f39512eda3aa0702b13ffed63')]

In [42]:
df_set3

,game_id,timestamp,ai5,event
8,18121481,2016-05-09 02:00:00.105905,e8ebfc3f39512eda3aa0702b13ffed63,ggstart
54,18121481,2016-05-09 02:00:01.724547,e8ebfc3f39512eda3aa0702b13ffed63,ggstop
483,18121481,2016-05-09 02:00:12.798309,e8ebfc3f39512eda3aa0702b13ffed63,ggstart
9671,18121481,2016-05-09 02:04:04.279836,e8ebfc3f39512eda3aa0702b13ffed63,ggstop
9783,18121481,2016-05-09 02:04:06.763143,e8ebfc3f39512eda3aa0702b13ffed63,ggstart
11161,18121481,2016-05-09 02:04:43.965911,e8ebfc3f39512eda3aa0702b13ffed63,ggstop
11179,18121481,2016-05-09 02:04:44.434762,e8ebfc3f39512eda3aa0702b13ffed63,ggstop
11188,18121481,2016-05-09 02:04:44.626066,e8ebfc3f39512eda3aa0702b13ffed63,ggstart
11635,18121481,2016-05-09 02:04:55.207604,e8ebfc3f39512eda3aa0702b13ffed63,ggstart
34175,18121481,2016-05-09 02:13:59.359081,e8ebfc3f39512eda3aa0702b13ffed63,ggstop


In [43]:
calc(df_set3)

//anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
//anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
//anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2387: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

[818.0]

### All the three sets give correct result.

# LET'S GROUP OUR DATA BASED ON (GAME_ID, AI5):

In [ ]:
# If we run the below function. We'll get the list of sessions 
def main_dict_function(df_analysis):
    #MAKING GROUPING OF DATAFRAME BASED ON GAME_ID AND DEVICE_ID-
    df_ana_group = df_analysis.groupby(['game_id','ai5'])
    return_dict = df_ana_group.apply(lambda x:calc(x))
    return return_dict

## From this main dict: we can do any sort of analysis for game_id or user_id.


### I didn't do the remaining part as above computation is quite heavy. But from here our first 2 problems are straight forward.